<a href="https://colab.research.google.com/github/licialearn/Crop_Analysis/blob/main/CropAnalysisModel_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mindspore -f https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.5.0/MindSpore/cpu/ubuntu_x86/mindspore.html

Looking in links: https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.5.0/MindSpore/cpu/ubuntu_x86/mindspore.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.0/962.0 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 92.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import mindspore as ms
import numpy as np

# 1. Load your dataset
df = pd.read_csv("/content/sample_data/agriculture_dataset.csv")

# 2. Drop columns that are not needed for prediction
drop_columns = [
    'High_Resolution_RGB', 'Multispectral_Images', 'Thermal_Images', 'Temporal_Images',
    'GPS_Coordinates', 'Field_Boundaries', 'Pest_Hotspots', 'Ground_Truth_Segmentation',
    'Bounding_Boxes', 'Crop_Health_Label'
]
df_model = df.drop(columns=drop_columns)

# 3. Encode the target column: Crop_Type
label_encoder = LabelEncoder()
df_model['Crop_Type'] = label_encoder.fit_transform(df_model['Crop_Type'])

# 4. Separate features and labels
X = df_model.drop(columns=['Crop_Type'])
y = df_model['Crop_Type']

# 5. Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 6. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# 7. Convert to MindSpore tensors
X_train_tensor = ms.Tensor(X_train, dtype=ms.float32)
y_train_tensor = ms.Tensor(y_train.to_numpy(), dtype=ms.int32)
X_test_tensor = ms.Tensor(X_test, dtype=ms.float32)
y_test_tensor = ms.Tensor(y_test.to_numpy(), dtype=ms.int32)

# Optional: Show data shapes
print("Train shape:", X_train_tensor.shape, y_train_tensor.shape)
print("Test shape:", X_test_tensor.shape, y_test_tensor.shape)


Train shape: (169615, 21) (169615,)
Test shape: (42404, 21) (42404,)


**Let’s now build and train a MindSpore neural network model to predict the Crop_Type**

In [18]:
import mindspore as ms
import mindspore.dataset as ds
import mindspore.nn as nn
import numpy as np
from mindspore import Model, Tensor
from mindspore.nn import SoftmaxCrossEntropyWithLogits, Momentum, Accuracy
from mindspore.train.callback import LossMonitor

# ============================================
# 1. Prepare Data
# ============================================
X_train_np = X_train_tensor.asnumpy()
y_train_np = y_train_tensor.asnumpy()
X_test_np = X_test_tensor.asnumpy()
y_test_np = y_test_tensor.asnumpy()

batch_size = 32

train_dataset = ds.NumpySlicesDataset(
    {"features": X_train_np, "labels": y_train_np}, shuffle=True
).batch(batch_size)

test_dataset = ds.NumpySlicesDataset(
    {"features": X_test_np, "labels": y_test_np}, shuffle=False
).batch(batch_size)

# ============================================
# 2. Define Model
# ============================================
class CropNet(nn.Cell):
    def __init__(self, input_dim, output_dim):
        super(CropNet, self).__init__()
        self.fc = nn.SequentialCell([
            nn.Dense(input_dim, 64),
            nn.ReLU(),
            nn.Dense(64, 32),
            nn.ReLU(),
            nn.Dense(32, output_dim)
        ])

    def construct(self, x):
        return self.fc(x)

input_dim = X_train_np.shape[1]
num_classes = len(np.unique(y_train_np))
net = CropNet(input_dim=input_dim, output_dim=num_classes)

# ============================================
# 3. Compile Training Setup
# ============================================
loss_fn = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
optimizer = Momentum(net.trainable_params(), learning_rate=0.01, momentum=0.9)
model = Model(net, loss_fn=loss_fn, optimizer=optimizer, metrics={"Accuracy": Accuracy()})

# ============================================
# 4. Train
# ============================================
print("🚀 Training...")
model.train(
    epoch=20,
    train_dataset=train_dataset,
    callbacks=[LossMonitor()],
    dataset_sink_mode=False
)

# ============================================
# 5. Evaluate
# ============================================
print("🧪 Evaluating...")
accuracy = model.eval(test_dataset, dataset_sink_mode=False)
print("✅ Test Accuracy:", accuracy)





Streaming output truncated to the last 5000 lines.
epoch: 20 step: 304, loss is 0.9320175051689148
epoch: 20 step: 305, loss is 0.989750325679779
epoch: 20 step: 306, loss is 1.0826568603515625
epoch: 20 step: 307, loss is 0.8756831884384155
epoch: 20 step: 308, loss is 0.9403860569000244
epoch: 20 step: 309, loss is 0.9436812996864319
epoch: 20 step: 310, loss is 0.9298222661018372
epoch: 20 step: 311, loss is 0.8649150729179382
epoch: 20 step: 312, loss is 0.8255336284637451
epoch: 20 step: 313, loss is 0.9096280336380005
epoch: 20 step: 314, loss is 0.8864724636077881
epoch: 20 step: 315, loss is 0.9062723517417908
epoch: 20 step: 316, loss is 0.7365908026695251
epoch: 20 step: 317, loss is 0.8825295567512512
epoch: 20 step: 318, loss is 1.086747169494629
epoch: 20 step: 319, loss is 0.8463107347488403
epoch: 20 step: 320, loss is 0.8600242733955383
epoch: 20 step: 321, loss is 0.9896159768104553
epoch: 20 step: 322, loss is 0.9727312326431274
epoch: 20 step: 323, loss is 1.12998187

In [2]:
#Step 1: Define the Neural Network
import mindspore.nn as nn

# Define a simple feedforward neural network
class CropNet(nn.Cell):
    def __init__(self, input_dim, output_dim):
        super(CropNet, self).__init__()
        self.fc = nn.SequentialCell(
            nn.Dense(input_dim, 64),
            nn.ReLU(),
            nn.Dense(64, 32),
            nn.ReLU(),
            nn.Dense(32, output_dim)
        )

    def construct(self, x):
        return self.fc(x)


In [3]:
#Step 2: Set Up the Training Pipeline
from mindspore import Model
from mindspore.nn import SoftmaxCrossEntropyWithLogits, Accuracy, Momentum
from mindspore.train.callback import LossMonitor

# Input and output dimensions
input_dim = X_train_tensor.shape[1]
num_classes = len(np.unique(y_train_tensor.asnumpy()))

# Create model
net = CropNet(input_dim=input_dim, output_dim=num_classes)

# Loss, optimizer, and metrics
loss_fn = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
optimizer = Momentum(net.trainable_params(), learning_rate=0.01, momentum=0.9)
model = Model(net, loss_fn=loss_fn, optimizer=optimizer, metrics={"Accuracy": Accuracy()})


In [7]:
print("X_train_tensor shape:", X_train_tensor.shape)
print("After asnumpy():", X_train_tensor.asnumpy().shape)

X_train_tensor shape: (169615, 21)
After asnumpy(): (169615, 21)


In [20]:
#Step 3: Train the Model

# Train for 1 epochs
model.train(
    epoch=1,
    train_dataset=train_dataset,
    callbacks=[LossMonitor()],
    dataset_sink_mode=False
)

Streaming output truncated to the last 5000 lines.
epoch: 1 step: 302, loss is 0.9726265072822571
epoch: 1 step: 303, loss is 0.7965282201766968
epoch: 1 step: 304, loss is 0.7968557476997375
epoch: 1 step: 305, loss is 0.8813217878341675
epoch: 1 step: 306, loss is 0.8237060904502869
epoch: 1 step: 307, loss is 1.0256707668304443
epoch: 1 step: 308, loss is 0.7984513640403748
epoch: 1 step: 309, loss is 0.7822090983390808
epoch: 1 step: 310, loss is 0.8958194851875305
epoch: 1 step: 311, loss is 0.8872965574264526
epoch: 1 step: 312, loss is 0.7342862486839294
epoch: 1 step: 313, loss is 0.9370300769805908
epoch: 1 step: 314, loss is 1.010238766670227
epoch: 1 step: 315, loss is 0.9975439310073853
epoch: 1 step: 316, loss is 0.799129068851471
epoch: 1 step: 317, loss is 0.676120936870575
epoch: 1 step: 318, loss is 0.9615422487258911
epoch: 1 step: 319, loss is 0.7449369430541992
epoch: 1 step: 320, loss is 0.8294854760169983
epoch: 1 step: 321, loss is 0.8970722556114197
epoch: 1 ste